In [4]:
    def _setup_tools(self, tools: List[Union[Dict[str, Any], Callable]]) -> None:
        """
        Process and set up tools for function calling.
        
        Args:
            tools: List of tools (functions or schema definitions)
        """
        for tool in tools:
            if callable(tool):
                # Convert callable to function definition
                function_def = self.utils.python_to_gemini_function(tool)
                self.tool_definitions.append(function_def)
                self.tool_callables[tool.__name__] = tool
            elif isinstance(tool, dict):
                # Already a function definition
                self.tool_definitions.append(tool)
                # Get the name for execution
                if "name" in tool:
                    name = tool["name"]
                    if name in self.tool_callables:
                        logger.warning(f"Duplicate tool name: {name}")
            else:
                logger.warning(f"Ignoring unsupported tool type: {type(tool)}")
    
    def _execute_tool(self, function_name: str, arguments: Dict[str, Any]) -> Any:
        """
        Execute a function from the registered tools.
        
        Args:
            function_name: Name of the function to execute
            arguments: Arguments to pass to the function
            
        Returns:
            Function result or error message
        """
        if function_name not in self.tool_callables:
            error_msg = f"Function '{function_name}' not found in registered tools"
            logger.error(error_msg)
            return {"error": error_msg}
            
        try:
            func = self.tool_callables[function_name]
            logger.info(f"Executing function: {function_name} with arguments: {arguments}")
            result = func(**arguments)
            return result
        except Exception as e:
            error_msg = f"Error executing function '{function_name}': {str(e)}"
            logger.error(error_msg)
            return {"error": error_msg}
    
    def _execute_function_calls(self, function_calls: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """
        Execute a list of function calls.
        
        Args:
            function_calls: List of function calls from model
            
        Returns:
            List of function results
        """
        results = []
        
        for call in function_calls:
            # Extract function name and arguments based on format
            name = call.get("name")
            if not name:
                name = call.get("functionCall", {}).get("name")
                
            # Get arguments, handling different formats
            args = call.get("args", {})
            if not args:
                args_raw = call.get("arguments") or call.get("functionCall", {}).get("arguments")
                if isinstance(args_raw, str):
                    try:
                        args = json.loads(args_raw)
                    except json.JSONDecodeError:
                        args = {}
                else:
                    args = args_raw or {}
            
            if not name:
                logger.warning("Function call missing name")
                continue
                
            # Execute the function
            result = self._execute_tool(name, args)
            
            # Add to results
            results.append({
                "name": name,
                "args": args,
                "result": result
            })
            
        return results
    
    
    def inference_with_tools(
        self,
        question: str,
        sso: str,
        schema: Optional[Dict[str, Any]] = None,
        document_id: Optional[str] = None,
        document_type: Optional[Literal["pdf", "jpg", "png", "jpeg"]] = None,
    ) -> Any:
        """
        Make an inference request with tool execution in two steps:
        1. First get function calls from model
        2. Execute functions
        3. Second inference with function results
        
        Args:
            question: User question or prompt
            sso: User's sso cookie
            schema: Optional response schema
            document_id: Optional document reference ID
            document_type: Optional document type
            
        Returns:
            Final model response with function results
        """
        # Early return if no tools
        if not self.tool_definitions:
            return self.inference(question, sso, schema, document_id, document_type)
            
        # Step 1: Get function calls
        logger.info("Step 1: Getting function calls from model")
        response = self.inference_service.cms_inference(
            question=question,
            sso=sso,
            tools=self.tool_definitions,
            document_id=document_id,
            document_type=document_type,
        )
        
        # Extract function calls
        function_calls = self.utils.extract_function_calls(response)
        
        # If no function calls, return the response
        if not function_calls:
            return response
            
        # Step 2: Execute functions
        logger.info(f"Step 2: Executing {len(function_calls)} function calls")
        function_results = self._execute_function_calls(function_calls)
        
        # Step 3: Get final response with function results
        logger.info("Step 3: Getting final response with function results")
        final_response = self.inference_service.cms_inference(
            question=question,
            sso=sso,
            schema=schema,
            document_id=document_id,
            document_type=document_type,
            function_results=function_results,
        )
        
        return final_response

    async def async_inference_with_tools(
        self,
        question: str,
        sso: str,
        schema: Optional[Dict[str, Any]] = None,
        document_id: Optional[str] = None,
        document_type: Optional[Literal["pdf", "jpg", "png", "jpeg"]] = None,
    ) -> Any:
        """
        Asynchronously make an inference request with tool execution in two steps.
        
        Args:
            question: User question or prompt
            sso: User's sso cookie
            schema: Optional response schema
            document_id: Optional document reference ID
            document_type: Optional document type
            
        Returns:
            Final model response with function results
        """
        # Early return if no tools
        if not self.tool_definitions:
            return await self.async_inference(question, sso, schema, document_id, document_type)
            
        # Step 1: Get function calls
        logger.info("Step 1: Getting function calls from model (async)")
        response = await self.inference_service.async_cms_inference(
            question=question,
            sso=sso,
            tools=self.tool_definitions,
            document_id=document_id,
            document_type=document_type,
        )
        
        # Extract function calls
        function_calls = self.utils.extract_function_calls(response)
        
        # If no function calls, return the response
        if not function_calls:
            return response
            
        # Step 2: Execute functions
        logger.info(f"Step 2: Executing {len(function_calls)} function calls")
        function_results = self._execute_function_calls(function_calls)
        
        # Step 3: Get final response with function results
        logger.info("Step 3: Getting final response with function results (async)")
        final_response = await self.inference_service.async_cms_inference(
            question=question,
            sso=sso,
            schema=schema,
            document_id=document_id,
            document_type=document_type,
            function_results=function_results,
        )
        
        return final_response